In [228]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import json
from time import sleep

In [229]:
headers={
        "accept-language": "en-US,en;q=0.9",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "accept-language": "en-US;en;q=0.9",
        "accept-encoding": "gzip, deflate, br",
    }

#url = 'https://www.zillow.com/rockville-md/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-77.36944914453125%2C%22east%22%3A-76.92450285546875%2C%22south%22%3A39.02324660629151%2C%22north%22%3A39.18469662654337%7D%2C%22usersSearchTerm%22%3A%22Rockville%2C%20MD%22%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A33714%2C%22regionType%22%3A6%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%7D'
url = 'https://www.zillow.com/rockville-md/2_p/'
req = requests.get(url, headers=headers)

In [230]:
def scrape_page(req):
    
    
    soup = bs(req.content)
    data = soup.find('script', {'id':'__NEXT_DATA__'})
    data = json.loads(data.text)
    data = data['props']['pageProps']['searchPageState']['cat1']['searchResults']['listResults'] # This is one page of results
    
    df = pd.json_normalize(data)

    return df


In [231]:
req = requests.get(url, headers=headers)

def scrape_results():
    dfs = []
    page_num = 1
    url = f'https://www.zillow.com/rockville-md/{page_num}_p/'
    req = requests.get(url, headers=headers)
    
    # Messy code - clean it up
    while req.status_code == 200:
        df = scrape_page(req)
        dfs.append(df)
        sleep(5)
        page_num += 1
        
        url = f'https://www.zillow.com/rockville-md/{page_num}_p/'
        req = requests.get(url, headers=headers)

    
    print(f'Scraped {len(dfs)} pages')
    dfs = pd.concat(dfs, axis=0)
    #dfs = dfs.drop_duplicates()
    dfs = dfs.drop_duplicates(subset='zpid')
    dfs = dfs.reset_index(drop=True)
    return dfs

df = scrape_results()
print(f'Found {len(df)} records')

Scraped 3 pages
Found 85 records


In [232]:
def getNumberFromAddressStreet(addressStreet):
    number =  addressStreet.split(' ')[0]
    # Some houses have no street number
    if number.isnumeric():
        return int(number)
    return np.nan

def getStreetFromAddressStreet(addressStreet):
    addressStreet =  addressStreet.split(' ')
    # Some houses have no street number - return entire addressstreet
    if addressStreet[0].isnumeric():
        return ' '.join(addressStreet[1:])
    return ' '.join(addressStreet)

In [251]:
columns = ['hdpData.homeInfo.zpid','addressStreet', 'hdpData.homeInfo.unit', 'addressCity', 'addressState', 'addressZipcode','hdpData.homeInfo.homeType',  'beds', 'baths', 'area', 'pgapt',
           'hdpData.homeInfo.price', 'hdpData.homeInfo.homeStatus', 'hdpData.homeInfo.priceForHDP', 'hdpData.homeInfo.taxAssessedValue', 'zestimate', 'hdpData.homeInfo.rentZestimate',
           'hdpData.homeInfo.daysOnZillow','hdpData.homeInfo.isPreforeclosureAuction',  'brokerName', 'latLong.latitude', 'latLong.longitude',  
           'hasOpenHouse', 'openHouseStartDate', 'openHouseEndDate', 'builderName',
           'hdpData.homeInfo.listing_sub_type.is_newHome']

# Drop prefixes from columns
df = df[columns]
columns = [col.split('.')[-1] for col in columns]

df.columns = columns


# Format addresses
df['StreetNumber'] = df['addressStreet'].map(getNumberFromAddressStreet)
df['Street'] = df['addressStreet'].map(getStreetFromAddressStreet)

# Get streets with no unit name
streets = []
for idx, street in enumerate(df['Street']):
    unit = df['unit'][idx]
    if type(unit) == str:
        street = street[:len(street) - len(unit)]
    streets.append(street.strip())
    
df['Street'] = streets
df['openHouseStartDate'] = pd.to_datetime(df['openHouseStartDate'])
df['openHouseEndDate'] = pd.to_datetime(df['openHouseEndDate'])

# Datetime conversion
df['openHouseStartDate'] = pd.to_datetime(df['openHouseStartDate'])
df['openHouseEndDate'] = pd.to_datetime(df['openHouseEndDate'])

# Some cleaning
df = df[~df['addressStreet'].str.lower().str.contains('homes available soon')]
df = df[df['homeType'].str.lower() != 'lot']


KeyError: "['hdpData.homeInfo.zpid', 'hdpData.homeInfo.unit', 'hdpData.homeInfo.homeType', 'hdpData.homeInfo.price', 'hdpData.homeInfo.homeStatus', 'hdpData.homeInfo.priceForHDP', 'hdpData.homeInfo.taxAssessedValue', 'hdpData.homeInfo.rentZestimate', 'hdpData.homeInfo.daysOnZillow', 'hdpData.homeInfo.isPreforeclosureAuction', 'latLong.latitude', 'latLong.longitude', 'hdpData.homeInfo.listing_sub_type.is_newHome'] not in index"

,zpid,addressStreet,unit,addressCity,addressState,addressZipcode,homeType,beds,baths,area,...,brokerName,latitude,longitude,hasOpenHouse,openHouseStartDate,openHouseEndDate,builderName,is_newHome,StreetNumber,Street
0,37334951,15414 Manor House Ter,NaN,Rockville,MD,20853,TOWNHOUSE,2.0,4.0,2070.0,...,"EXP Realty, LLC",39.108810,-77.078000,NaN,NaT,NaT,NaN,NaN,15414.0,Manor House Ter
1,97483084,613 Azalea Dr,NaN,Rockville,MD,20850,TOWNHOUSE,2.0,3.0,1656.0,...,Compass,39.094560,-77.174000,NaN,NaT,NaT,NaN,NaN,613.0,Azalea Dr
2,37252654,10010 Vanderbilt Cir #129,# 129,Rockville,MD,20850,CONDO,3.0,2.0,1486.0,...,Keller Williams Integrity,39.109850,-77.209390,True,2023-09-24 13:00:00,2023-09-24 15:00:00,NaN,NaN,10010.0,Vanderbilt Cir
3,97460406,623 Azalea Dr APT 1,Apt 1,Rockville,MD,20850,CONDO,1.0,1.0,1150.0,...,"Long & Foster Real Estate, Inc.",39.094390,-77.174515,NaN,NaT,NaT,NaN,NaN,623.0,Azalea Dr
4,166622926,16662 Crabbs Branch Way,NaN,Rockville,MD,20855,TOWNHOUSE,3.0,3.0,1524.0,...,KW Metro Center,39.124590,-77.164590,True,2023-09-22 12:00:00,2023-09-22 16:00:00,Lennar,True,16662.0,Crabbs Branch Way
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,2056440082,"Homes Available Soon, The Grove",# GIVWDF,Rockville,MD,20850,MULTI_FAMILY,NaN,0.0,NaN,...,NaN,39.096620,-77.203760,NaN,NaT,NaT,Craftmark Homes,True,NaN,"Homes Available Soon, T"
79,2059767876,8235 Front Loop,NaN,Derwood,MD,20855,TOWNHOUSE,4.0,5.0,2965.0,...,Coldwell Banker Realty,39.126970,-77.168976,NaN,NaT,NaT,Lennar Homes,True,8235.0,Front Loop
81,82140304,38 Maryland Ave UNIT 605,Unit 605,Rockville,MD,20850,CONDO,1.0,1.0,993.0,...,Compass,39.085320,-77.150590,NaN,NaT,NaT,NaN,NaN,38.0,Maryland Ave
82,37106155,2005 Mac Dowell St,NaN,Rockville,MD,20851,SINGLE_FAMILY,4.0,3.0,1734.0,...,"Spring Hill Real Estate, LLC.",39.077090,-77.116260,NaN,NaT,NaT,NaN,NaN,2005.0,Mac Dowell St


In [237]:
df['StreetNumber'].astype(int)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [235]:
df.to_csv('clean.csv', index=False)